In [1]:
import pandas as pd
from pandas import json_normalize
import requests
import json
import snowflake.connector

from dotenv import load_dotenv
import os
import http.client
import urllib.parse


from googletrans import Translator
import string
import re

from sqlalchemy import create_engine

In [2]:
load_dotenv()
rapidapi_key = os.getenv('RAPIDAPI_KEY')
rapidapi_host = "linkedin-job-search-api.p.rapidapi.com"
snowflake_password = os.getenv('SNOWFLAKE_PASSWORD')

In [3]:
def extract_linkedin_job_data():

    headers = {
        'x-rapidapi-key': rapidapi_key,
        'x-rapidapi-host': rapidapi_host
    }

    location = "Australia"
    limit = 100
    offset = 0
    titles = ["Data Engineer" , "Data Scientist", "Data Analyst"]

    df_daily_all = pd.DataFrame()

    for title_filter in titles:

        #URL code the title filter and location filter
        title_encoded=urllib.parse.quote(title_filter)
        location_encoded=urllib.parse.quote(location)

        #API endpoint 7day data query
        #base_url = f"/active-jb-7d?limit={limit}&offset={offset}&title_filter={title_encoded}&location_filter={location_encoded}"

        #API endpoint last 24 hour data query
        base_url = f"/active-jb-24h?limit={limit}&offset={offset}&title_filter={title_encoded}&location_filter={location_encoded}"
        url =  f"https://{rapidapi_host}{base_url}"


        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            # Convert the JSON data to a DataFrame
            df_daily = json_normalize(data)
            df_daily['job_category'] = title_filter
            print(title_filter, df_daily.shape)
            df_daily_all = pd.concat([df_daily_all, df_daily], ignore_index=True)


        else:
            print(f"Error: {response.status_code}")

    return df_daily_all

df_daily_all = extract_linkedin_job_data()
# df_daily_all

Data Engineer (33, 51)
Data Scientist (4, 51)
Data Analyst (11, 51)


In [10]:
# df_daily_all = pd.read_csv('linkedin_jobs_daily.csv')
df_daily_all.head()

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
2,1613938297,2025-05-02T11:21:35,2025-05-02T11:37:46.92682,Full Stack Developer/ Senior Data Engineer,Data Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-06-01T11:21:35,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Sydney, NSW",Privately Held,1990.0,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank
4,1613760344,2025-05-02T09:55:55,2025-05-02T10:30:56.886326,Senior Data Engineer (Hadoop/ Teradata Platform),Data Engineer,Commonwealth Bank,https://www.linkedin.com/company/commonwealthbank,2025-06-18T02:07:28,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Sydney, NSW",Public Company,NaN,[''],"['., Sydney, NSW 2000, AU']",Australia’s leading provider of financial serv...,False,Mid-Senior level,False,commonwealthbank
5,1613556983,2025-05-02T08:48:08,2025-05-02T09:03:16.40536,Data Engineer,Data Engineer,UCentric,https://www.linkedin.com/company/ucentricpeople,2025-06-01T08:48:08,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Sydney, New South Wales",Privately Held,2024.0,"['executive search', 'technology recruitment',...","['Sydney, New South Wales 2000, AU', 'Palo Alt...","At UCentric®, you are at the core of everythin...",True,Associate,True,ucentricpeople
6,1613557663,2025-05-02T08:38:41,2025-05-02T09:03:31.847953,Principal Data Engineer,Data Engineer,LocalAgentFinder,https://www.linkedin.com/company/localagentfinder,2025-06-01T08:38:40,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"St Kilda, VIC",Privately Held,2007.0,['Real Estate Agent Fees and Real Estate Agent...,"['14 St Kilda Road, St Kilda, VIC 3182, AU']",LocalAgentFinder empowers homeowners and prope...,False,Mid-Senior level,True,localagentfinder
7,1613463440,2025-05-02T08:19:55,2025-05-02T08:31:18.93876,Data Engineer,Data Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-06-01T08:16:00,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Sydney, NSW",Privately Held,1990.0,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Not Applicable,False,peoplebank


In [11]:
def get_clean_data_jobs(df_daily_all):
     pattern=re.compile(r'\bData Engineer\b|\bData Scientist\b|\bData Analyst\b' , re.IGNORECASE)
     df_daily_all = df_daily_all[df_daily_all['TITLE'].str.contains(pattern)]
     return df_daily_all

df_daily_all = get_clean_data_jobs(df_daily_all)
df_daily_all.shape

(38, 44)

In [12]:
df_daily_all.to_csv('linkedin_jobs_daily.csv', index=False)

In [13]:
df_daily_all = pd.read_csv('linkedin_jobs_daily.csv')
df_daily_all.shape

(38, 44)

In [14]:
def update_columns(df_daily_all):
    df_daily_all.columns = df_daily_all.columns.str.upper()
    df_daily_all = df_daily_all[['ID', 'DATE_POSTED', 'DATE_CREATED', 'TITLE', 'JOB_CATEGORY',
       'ORGANIZATION', 'ORGANIZATION_URL', 'DATE_VALIDTHROUGH', 'LOCATIONS_RAW',
       'LOCATION_TYPE', 'LOCATION_REQUIREMENTS_RAW', 'EMPLOYMENT_TYPE', 'URL',
       'SOURCE_TYPE', 'SOURCE', 'SOURCE_DOMAIN', 'ORGANIZATION_LOGO',
       'CITIES_DERIVED', 'REGIONS_DERIVED', 'COUNTRIES_DERIVED',
       'LOCATIONS_DERIVED', 'TIMEZONES_DERIVED', 'LATS_DERIVED',
       'LNGS_DERIVED', 'REMOTE_DERIVED', 'RECRUITER_NAME', 'RECRUITER_TITLE',
       'RECRUITER_URL', 'LINKEDIN_ORG_EMPLOYEES', 'LINKEDIN_ORG_URL',
       'LINKEDIN_ORG_SIZE', 'LINKEDIN_ORG_SLOGAN', 'LINKEDIN_ORG_INDUSTRY',
       'LINKEDIN_ORG_FOLLOWERS', 'LINKEDIN_ORG_HEADQUARTERS',
       'LINKEDIN_ORG_TYPE', 'LINKEDIN_ORG_FOUNDEDDATE',
       'LINKEDIN_ORG_SPECIALTIES', 'LINKEDIN_ORG_LOCATIONS',
       'LINKEDIN_ORG_DESCRIPTION', 'LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED',
       'SENIORITY', 'DIRECTAPPLY', 'LINKEDIN_ORG_SLUG']]
    return df_daily_all

df_daily_all = update_columns(df_daily_all)
df_daily_all.head(2)

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1613938297,2025-05-02T11:21:35,2025-05-02T11:37:46.92682,Full Stack Developer/ Senior Data Engineer,Data Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-06-01T11:21:35,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Sydney, NSW",Privately Held,1990.0,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank
1,1613760344,2025-05-02T09:55:55,2025-05-02T10:30:56.886326,Senior Data Engineer (Hadoop/ Teradata Platform),Data Engineer,Commonwealth Bank,https://www.linkedin.com/company/commonwealthbank,2025-06-18T02:07:28,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Sydney, NSW",Public Company,NaN,[''],"['., Sydney, NSW 2000, AU']",Australia’s leading provider of financial serv...,False,Mid-Senior level,False,commonwealthbank


### Process the df_daily_all data 


In [15]:
def extract_job_date(date_created):
    #extract the job_date from job_created
    df_daily_all['job_date'] = pd.to_datetime(df_daily_all['DATE_CREATED']).dt.date
    return df_daily_all['job_date']

#Apply the function to the DataFrame
df_daily_all['job_date'] = extract_job_date(df_daily_all['DATE_CREATED'])
df_daily_all['job_date'][:5]


0    2025-05-02
1    2025-05-02
2    2025-05-02
3    2025-05-02
4    2025-05-02
Name: job_date, dtype: object

In [16]:
#Extrac the job city and state from the LOCATIONS_RAW field

def extract_city(list):
    """
    Extracts the city from the given text using regex.
    """
    # Regex pattern to match the city in the LOCATIONS_RAW field
    text = str(list)
    city_pattern = r"'addressLocality':\s*'(.*)',\s'addressRegion':"
    match = re.search(city_pattern, text)
    if match:
        city = match.group(1)
        if 'sidney' in city.lower() or 'sídney' in city.lower() or '悉尼' in city.lower(): #fix wrong city name
            return "Sydney"
        return city
    else:
        return None



def extract_state(list):
    """
    Extracts the state from the given text using regex.
    """
    # Regex pattern to match the region/state in the LOCATIONS_RAW field
    text = str(list)
    state_pattern = r"'addressRegion':\s*(.*)',\s'streetAddress'"
    match = re.search(state_pattern, text)
    if match:
        state = match.group(1)
        state= state.replace("'", "").strip()
        return state
    else:
        return None


#Extract city and state from Locations Raw
df_daily_all['city'] = df_daily_all['LOCATIONS_RAW'].apply(extract_city)


df_daily_all['state'] = df_daily_all['LOCATIONS_RAW'].apply(extract_state)



df_daily_all.head()

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG,job_date,city,state
0,1613938297,2025-05-02T11:21:35,2025-05-02T11:37:46.92682,Full Stack Developer/ Senior Data Engineer,Data Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-06-01T11:21:35,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank,2025-05-02,Woden,None
1,1613760344,2025-05-02T09:55:55,2025-05-02T10:30:56.886326,Senior Data Engineer (Hadoop/ Teradata Platform),Data Engineer,Commonwealth Bank,https://www.linkedin.com/company/commonwealthbank,2025-06-18T02:07:28,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,[''],"['., Sydney, NSW 2000, AU']",Australia’s leading provider of financial serv...,False,Mid-Senior level,False,commonwealthbank,2025-05-02,Sydney,NSW
2,1613556983,2025-05-02T08:48:08,2025-05-02T09:03:16.40536,Data Engineer,Data Engineer,UCentric,https://www.linkedin.com/company/ucentricpeople,2025-06-01T08:48:08,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['executive search', 'technology recruitment',...","['Sydney, New South Wales 2000, AU', 'Palo Alt...","At UCentric®, you are at the core of everythin...",True,Associate,True,ucentricpeople,2025-05-02,Sydney,NSW
3,1613557663,2025-05-02T08:38:41,2025-05-02T09:03:31.847953,Principal Data Engineer,Data Engineer,LocalAgentFinder,https://www.linkedin.com/company/localagentfinder,2025-06-01T08:38:40,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,['Real Estate Agent Fees and Real Estate Agent...,"['14 St Kilda Road, St Kilda, VIC 3182, AU']",LocalAgentFinder empowers homeowners and prope...,False,Mid-Senior level,True,localagentfinder,2025-05-02,Melbourne,VIC
4,1613463440,2025-05-02T08:19:55,2025-05-02T08:31:18.93876,Data Engineer,Data Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-06-01T08:16:00,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Not Applicable,False,peoplebank,2025-05-02,Perth,WA


In [17]:
#Extract the employment type from the EMPLOYMENT_TYPE field

def extract_employment_type(df_daily_all):
    """
    Extracts the employment type from the given text using regex.
    """
    df_daily_all['employment_type'] = (
        df_daily_all['EMPLOYMENT_TYPE']
        .astype(str)
        .str.replace(r"[\[\]']", '', regex=True)
        .str.strip()
    )

    df_daily_all.drop(columns=['EMPLOYMENT_TYPE'], inplace=True)

    return df_daily_all['employment_type']


df_daily_all['employment_type'] = extract_employment_type(df_daily_all)
df_daily_all['employment_type'].tail()

33    FULL_TIME
34    FULL_TIME
35    FULL_TIME
36    FULL_TIME
37    FULL_TIME
Name: employment_type, dtype: object

In [18]:
#Extract the employment size
def extract_employee_size(LINKEDIN_ORG_SIZE):
    """
    Extracts the employee size from the given text using regex.
    """
    df_daily_all['org_size'] = (
        df_daily_all['LINKEDIN_ORG_SIZE']
        .astype(str)
        .str.replace(r"employees", '', regex=True)
        .str.strip()
    )

    return df_daily_all['org_size']


df_daily_all['org_size'] = extract_employee_size(df_daily_all['LINKEDIN_ORG_SIZE'])
df_daily_all['org_size'].tail()

33        10,001+
34      501-1,000
35    1,001-5,000
36         51-200
37      501-1,000
Name: org_size, dtype: object

In [19]:
df_daily_all.columns = df_daily_all.columns.str.upper()
df_daily_all.head(2)

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORG_SIZE
0,1613938297,2025-05-02T11:21:35,2025-05-02T11:37:46.92682,Full Stack Developer/ Senior Data Engineer,Data Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-06-01T11:21:35,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank,2025-05-02,Woden,None,CONTRACTOR,201-500
1,1613760344,2025-05-02T09:55:55,2025-05-02T10:30:56.886326,Senior Data Engineer (Hadoop/ Teradata Platform),Data Engineer,Commonwealth Bank,https://www.linkedin.com/company/commonwealthbank,2025-06-18T02:07:28,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,Australia’s leading provider of financial serv...,False,Mid-Senior level,False,commonwealthbank,2025-05-02,Sydney,NSW,FULL_TIME,"10,001+"


In [20]:
#Only keep the relevant columns

df_daily_all = df_daily_all[['ID', 'TITLE', 'JOB_CATEGORY',
       'JOB_DATE', 'CITY', 'STATE', 'EMPLOYMENT_TYPE' ,
       'ORGANIZATION', 'ORGANIZATION_URL', 'URL',
       'SOURCE_TYPE', 'SOURCE', 'SOURCE_DOMAIN',
       'ORGANIZATION_LOGO', 'REMOTE_DERIVED', 'RECRUITER_NAME', 'RECRUITER_TITLE',
       'RECRUITER_URL', 'LINKEDIN_ORG_URL',
       'ORG_SIZE', 'LINKEDIN_ORG_INDUSTRY',
       'LINKEDIN_ORG_HEADQUARTERS',
       'LINKEDIN_ORG_TYPE', 'LINKEDIN_ORG_FOUNDEDDATE',
       'LINKEDIN_ORG_SPECIALTIES', 'LINKEDIN_ORG_LOCATIONS',
       'LINKEDIN_ORG_DESCRIPTION','LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED',
       'SENIORITY', 'DIRECTAPPLY',
       'LINKEDIN_ORG_SLUG']]


df_daily_all.head()

,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1613938297,Full Stack Developer/ Senior Data Engineer,Data Engineer,2025-05-02,Woden,None,CONTRACTOR,Peoplebank,https://www.linkedin.com/company/peoplebank,https://au.linkedin.com/jobs/view/full-stack-d...,...,"Sydney, NSW",Privately Held,1990.0,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank
1,1613760344,Senior Data Engineer (Hadoop/ Teradata Platform),Data Engineer,2025-05-02,Sydney,NSW,FULL_TIME,Commonwealth Bank,https://www.linkedin.com/company/commonwealthbank,https://au.linkedin.com/jobs/view/senior-data-...,...,"Sydney, NSW",Public Company,NaN,[''],"['., Sydney, NSW 2000, AU']",Australia’s leading provider of financial serv...,False,Mid-Senior level,False,commonwealthbank
2,1613556983,Data Engineer,Data Engineer,2025-05-02,Sydney,NSW,FULL_TIME,UCentric,https://www.linkedin.com/company/ucentricpeople,https://au.linkedin.com/jobs/view/data-enginee...,...,"Sydney, New South Wales",Privately Held,2024.0,"['executive search', 'technology recruitment',...","['Sydney, New South Wales 2000, AU', 'Palo Alt...","At UCentric®, you are at the core of everythin...",True,Associate,True,ucentricpeople
3,1613557663,Principal Data Engineer,Data Engineer,2025-05-02,Melbourne,VIC,FULL_TIME,LocalAgentFinder,https://www.linkedin.com/company/localagentfinder,https://au.linkedin.com/jobs/view/principal-da...,...,"St Kilda, VIC",Privately Held,2007.0,['Real Estate Agent Fees and Real Estate Agent...,"['14 St Kilda Road, St Kilda, VIC 3182, AU']",LocalAgentFinder empowers homeowners and prope...,False,Mid-Senior level,True,localagentfinder
4,1613463440,Data Engineer,Data Engineer,2025-05-02,Perth,WA,FULL_TIME,Peoplebank,https://www.linkedin.com/company/peoplebank,https://au.linkedin.com/jobs/view/data-enginee...,...,"Sydney, NSW",Privately Held,1990.0,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Not Applicable,False,peoplebank


## Connect to  Snowflake database for raw data initial processing

In [21]:
#Establish a connection to Snowflake

def connect_to_snowflake():
    try:

        conn = snowflake.connector.connect(
            user="NIKKILW2025",
            password=snowflake_password,
            account="gbszkwp-by30611",
            warehouse="SNOWFLAKE_LEARNING_WH",
            database="linkedin_db",
            schema="linkedin_raw"
        )
        print("Connection to Snowflake established successfully.")
        return conn
    except Exception as e:
        print(f"Error connecting to Snowflake: {e}")
        return None

conn = connect_to_snowflake()

Connection to Snowflake established successfully.


In [22]:
#query the raw data and narrow down to DE, DS and DA roles
#This data is filtered by relevant roles, and translated into English (but no city, state and seniority fix)
def query_existing_job_data(conn):
    query = """
        SELECT * FROM LINKEDIN_JOB_API_CLEANED_DATA
        WHERE (
            lower(TITLE) LIKE '%data engineer%'
            OR lower(TITLE) LIKE '%data scientist%'
            OR lower(TITLE) LIKE '%data analyst%'
            )
    """

    df = pd.read_sql(query, conn)
    print(df.shape)
    return df

df = query_existing_job_data(conn)
df.head()

/tmp/ipykernel_11274/824833193.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


(258, 31)


,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1599037955,Senior Data Engineer,Data Engineer,2025-04-28,Greater Sydney Area,NA,FULL_TIME,One51 | Data & Analytics Consultancy,https://www.linkedin.com/company/one51consulting,https://au.linkedin.com/jobs/view/senior-data-...,...,"Sydney, NSW",Privately Held,2020.0,"['Business Intelligence', 'Data Warehousing', ...","['333 George Street, Level 13, Sydney, NSW 200...",Drawing on a wealth of expertise and a deep un...,False,Mid-Senior level,False,one51consulting
1,1598878880,Senior Data Engineer,Data Engineer,2025-04-28,Brisbane,QLD,CONTRACTOR,Data#3,https://www.linkedin.com/company/data3,https://au.linkedin.com/jobs/view/senior-data-...,...,"Toowong, Queensland",Public Company,1977.0,"['Cloud Solutions', 'Mobility Solutions', 'Sec...","['555 Coronation Dr, Toowong, Queensland 4066,...","Data#3 Limited (DTL), is focused on helping cu...",False,Medium-high level,False,data3
2,1598880069,Data Engineer,Data Engineer,2025-04-28,Sydney,NSW,CONTRACTOR,Whizdom,https://www.linkedin.com/company/whizdom-recru...,https://au.linkedin.com/jobs/view/data-enginee...,...,"Canberra, Australian Capital Territory",Privately Held,2006.0,"['IT Recruitment', 'Recruitment for Government...","['28-34 Thynne St, Unit 7, Canberra, Australia...","Established in 2006, Whizdom is an Australian ...",True,Middle level,True,whizdom-recruitment
3,1598878894,Senior Data Engineer and Business Intelligence...,Data Engineer,2025-04-28,Moreton Bay,QLD,FULL_TIME,University of the Sunshine Coast,https://www.linkedin.com/school/university-of-...,https://au.linkedin.com/jobs/view/senior-data-...,...,None,None,NaN,None,None,None,None,Not Applicable,False,None
4,1598880083,Senior Data Engineer and Business Intelligence...,Data Engineer,2025-04-28,Sunshine Coast,QLD,FULL_TIME,University of the Sunshine Coast,https://www.linkedin.com/school/university-of-...,https://au.linkedin.com/jobs/view/senior-data-...,...,None,None,NaN,None,None,None,None,Not Applicable,False,None


In [23]:
df.to_csv('api_cleaned_job_id_snowflake.csv', index=False)

In [24]:
list_daily = df_daily_all.columns.tolist()
list_df = df.columns.tolist()
list_daily ==  list_df

True

In [25]:
#Check the Job ID from df and only keep those new jobs based on the Job IDs
def keep_new_jobs():
    existing_job_ids = df.ID.unique().tolist()

    df_new_jobs = df_daily_all[~(df_daily_all['ID'].isin(existing_job_ids))].reset_index(drop=True)
    print(f'{df_new_jobs.shape[0]} new jobs ready to load to Snowflake')
    return df_new_jobs

df_new_jobs = keep_new_jobs()
df_new_jobs.tail()

18 new jobs ready to load to Snowflake


,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
13,1613858663,Data Analyst (0 Experience Required),Data Analyst,2025-05-02,Melbourne,VIC,CONTRACTOR,Peroptyx,https://www.linkedin.com/company/peroptyx,https://au.linkedin.com/jobs/view/data-analyst...,...,"Castlebar, County Mayo",Privately Held,2018.0,"['Model Evaluation', 'Ground Truth', 'Global P...","['Innovation House, Moneen Road, Castlebar, Co...",Peroptyx is a provider of location-based machi...,False,Entry level,False,peroptyx
14,1613455968,Senior Data Analyst (AU),Data Analyst,2025-05-02,Melbourne,VIC,FULL_TIME,AKQA,https://www.linkedin.com/company/akqa,https://au.linkedin.com/jobs/view/senior-data-...,...,"San Francisco, CA",Public Company,NaN,"['Design and Innovation', 'Customer and Brand ...","['5th floor, 360 Third St, San Francisco, CA 9...",AKQA is a design and innovation company that e...,False,Orta-Üst Düzey Yönetici,False,akqa
15,1613213960,Data Analyst - Workforce DEI,Data Analyst,2025-05-02,Footscray,VIC,FULL_TIME,Western Health,https://www.linkedin.com/company/western-health,https://au.linkedin.com/jobs/view/data-analyst...,...,"Footscray, Victoria",Nonprofit,1953.0,[''],"['Gordon Street, Footscray, Victoria 3011, AU'...",Western Health is proud to provide a combinati...,False,Entry level,False,western-health
16,1613145714,Data Analyst,Data Analyst,2025-05-02,Brisbane,QLD,FULL_TIME,Davidson,https://www.linkedin.com/company/davidsongroup,https://au.linkedin.com/jobs/view/data-analyst...,...,"Brisbane City, Queensland",Privately Held,1991.0,"['Corporate', 'Projects & Operations', 'HR out...","['240 Queen St, Brisbane City, Queensland 4000...","As a proudly Australian company, over 30 years...",True,Junior,False,davidsongroup
17,1614358573,Master Data Analyst,Data Analyst,2025-05-02,Walkervale,QLD,FULL_TIME,Sunwater,https://www.linkedin.com/company/sunwater,https://au.linkedin.com/jobs/view/master-data-...,...,"Fortitude Valley, Queensland",Government Agency,2000.0,['Water Infrastructure'],"['515 Saint Pauls Terrace, Level 9, Fortitude ...",Sunwater is a water solutions and service prov...,False,Pleno-sênior,False,sunwater


In [ ]:
#Translate new job's business name, city, state and seniority to English

translator = Translator()

def translate_text(text, target_language='en'):
    try:
        if not text or pd.isna(text):
            return 'NA'
        translated_text = translator.translate(str(text), dest=target_language)
        return translated_text.text
    except Exception as e:
        print(f"Error translating text: {e} (Text: {text})")
        return text

for col in ['CITY', 'STATE', 'ORGANIZATION', 'SENIORITY']:
    unique_values = df_new_jobs[col].dropna().unique()
    translation_map = {val: translate_text(val, target_language='en') for val in unique_values}
    df_new_jobs[col] = df_new_jobs[col].map(translation_map).fillna('NA')

Error translating text: list index out of range (Text: pay.com.au)


In [ ]:
df_new_jobs['CITY'].unique()

array(['Woden', 'Sydney', 'Melbourne', 'Perth', 'Macquarie Park',
       'Docklands', 'Brisbane', 'Footscray', 'Walkervale'], dtype=object)

In [29]:
df_new_jobs['STATE'].unique()

array(['Na', 'NSW', 'VIC', 'Of', 'QLD'], dtype=object)

In [30]:

def load_to_snowflake(df_new_jobs):
    # Create a Snowflake connection engine
    engine = create_engine(
        'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'.format(
            user="NIKKILW2025",
            password=snowflake_password,
            account="gbszkwp-by30611",
            warehouse="SNOWFLAKE_LEARNING_WH",
            database="linkedin_db",
            schema="linkedin_raw"
        )
    )

    table_name = "linkedin_job_api_cleaned_data"

    df_new_jobs.to_sql(
        name=table_name,
        con=engine,
        if_exists='append', #append data
        index=False
    )

    print(f"Data loaded to Snowflake table {table_name} successfully.")


load_to_snowflake(df_new_jobs)

Data loaded to Snowflake table linkedin_job_api_cleaned_data successfully.
